# Flow 02_📍_BusLocator.py
I'm selecting route 16. Ballinteer - Dublin Airport towards Ballinter.

In [1]:
#Importing required libraries:
import os
import requests
import pandas as pd
import pytz  # For timezone handling
from datetime import datetime, timedelta
# API Key from environment variable
YOUR_API_KEY = os.environ.get("API_KEY")

1) Dropdown:Select a Route:

In [2]:
# Read the routes.txt file
file_path = "../data/Routes.txt" 
cr_df = pd.read_csv(file_path)
cr_df.head(20)

,route_id,incoming_bus,route_name
0,4451_86188,1,1. Sandymount - O'Connell Street - Santry
1,4451_86109,4,4. Monkstown Avenue - Harristown
2,4451_86107,6,6. Howth Dart Stn - Lower Abbey Street
3,4451_86151,7,7. Brides Glen Luas - Mountjoy Square
4,4451_86152,7A,7A. Loughlinstown - Mountjoy Sq
5,4451_86153,7B,7B. Shankill - Mountjoy Square
6,4451_86154,7D,7D. Dalkey - Mountjoy Square
7,4451_86155,7E,7E. Mountjoy Square - Dalkey
8,4451_86183,9,9. Limekiln Avenue - Charlestown
9,4451_86108,11,11. Sandyford Ind Estate - Wadelai Pk


In [3]:
route_name = '16. Ballinteer - Dublin Airport'
route_id = '4451_86113'

2) Dropdown: Towards

In [4]:
# Read the Towards.txt file
file_path = "../data/Towards.txt" 
sd_df = pd.read_csv(file_path)
sd_df.head(20)

,route_id,trip_id,trip_headsign,direction_id
0,4451_86107,4451_12191,Abbey St Lower,1
1,4451_86107,4451_12134,Howth Station,0
2,4451_86108,4451_444,O'Connell Street,0
3,4451_86108,4451_343,Sandyford B.D.,0
4,4451_86108,4451_447,St Pappin's Rd,1
5,4451_86108,4451_551,Parnell Square,1
6,4451_86109,4451_8619,Monkstown Ave,0
7,4451_86109,4451_8807,O'Connell St,0
8,4451_86109,4451_8810,Harristown,1
9,4451_86109,4451_8996,O'Connell St,1


Filtering by selected route 16. Ballinteer - Dublin Airport, below pd df would be the actual dropdown, creating variable trip_id

In [5]:
display = sd_df[(sd_df["route_id"] == route_id)]

display

,route_id,trip_id,trip_headsign,direction_id
18,4451_86113,4451_5781,O'Connell Street,1
19,4451_86113,4451_5554,Dublin Airport,1
20,4451_86113,4451_5545,O'Connell Street,0
21,4451_86113,4451_5330,Ballinteer,0


In [6]:
towards = '4451_5330'
trip_headsign = 'Ballinteer'

Filtering by the selected route and direction 16. Ballinteer - Dublin Airport towards Ballinteer

In [7]:

# Vehicles API Endpoint
VEHICLES_API_URL = "https://api.nationaltransport.ie/gtfsr/v2/Vehicles?format=json"

# Function to fetch Vehicles data with error handling
def fetch_vehicles_data():
    headers = {"x-api-key": YOUR_API_KEY}
    try:
        response = requests.get(VEHICLES_API_URL, headers=headers, verify=False)
        response.raise_for_status()  # Raise error for non-200 responses
        data = response.json()

        if "entity" not in data:
            print("⚠️ 'entity' key not found in Vehicles API response.")
            return None

        return data

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP Error: {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Request Error: {req_err}")
    except ValueError as json_err:
        print(f"JSON Decode Error: {json_err}")

    return None

# Fetch Vehicles data
vehicles_data = fetch_vehicles_data()

if vehicles_data is None:
    print("❌ No data fetched. Please check the API key or network connection.")
else:
    print("✅ Vehicles data fetched successfully!")

# Function to process and filter Vehicles data
def process_vehicles_data(data, concat):
    vehicles = []

    for entity in data.get("entity", []):
        vehicle_info = entity.get("vehicle", {})
        trip_info = vehicle_info.get("trip", {})
        position_info = vehicle_info.get("position", {})

        # Extract trip details
        trip_id = trip_info.get("trip_id")
        route_id = trip_info.get("route_id")
        direction_id = trip_info.get("direction_id")
        start_time = trip_info.get("start_time")
        start_date = trip_info.get("start_date")
        schedule_relationship = trip_info.get("schedule_relationship")
        vehicle_id = vehicle_info.get("vehicle", {}).get("id")
        latitude = position_info.get("latitude")
        longitude = position_info.get("longitude")

        # Skip if not scheduled
        if schedule_relationship != "SCHEDULED":
            continue

        # Combine route_id and direction_id
        combined_id = str(route_id) + str(direction_id)

        # Filter by the provided 'concat' value
        if combined_id == concat:
            vehicles.append({
                "trip_id": trip_id,
                "route_id": route_id,
                "direction_id": direction_id,
                "vehicle_id": vehicle_id,
                "start_time": start_time,
                "start_date": start_date,
                "latitude": latitude,
                "longitude": longitude
            })

    return pd.DataFrame(vehicles)

# Example 'concat' value (update this to match your filter)
concat = '4451_861130'  # Example: route_id + direction_id

# Process and filter the data
if vehicles_data:
    filtered_vehicles = process_vehicles_data(vehicles_data, concat).sort_values(by="start_time", ascending=True).reset_index(drop=True)

    if filtered_vehicles.empty:
        print("⚠️ No vehicles found for the specified 'concat' filter.")
    else:
        print("✅ Filtered vehicles data:")

/home/joserico/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.nationaltransport.ie'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ Vehicles data fetched successfully!
✅ Filtered vehicles data:


In [8]:
print("Buses operating route {} towards {}".format(route_name, trip_headsign))
filtered_vehicles

Buses operating route 16. Ballinteer - Dublin Airport towards Ballinteer


,trip_id,route_id,direction_id,vehicle_id,start_time,start_date,latitude,longitude
0,4451_5466,4451_86113,0,5296,16:45:00,20250119,53.271782,-6.248226
1,4451_5469,4451_86113,0,829,17:00:00,20250119,53.271800,-6.247683
2,4451_5473,4451_86113,0,888,17:15:00,20250119,53.312435,-6.282494
3,4451_5476,4451_86113,0,743,17:30:00,20250119,53.322105,-6.279385
4,4451_5480,4451_86113,0,1090,17:45:00,20250119,53.343445,-6.264402
5,4451_5483,4451_86113,0,666,18:00:00,20250119,53.354595,-6.263608
6,4451_5487,4451_86113,0,716,18:15:00,20250119,53.392750,-6.246017
